In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [7]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,27,19,6,2,64,25,39,63,2.33,59.9,32.9,26.9,1.00,54101,Mohamed Salah - 15,Alisson,NaN,60.4,60,2.22,48,1.78,33.3,6.52,0.32,34.1,2024
1,2,Manchester City,27,19,5,3,62,27,35,62,2.30,55.0,24.9,30.1,1.12,52996,Erling Haaland - 18,Ederson,NaN,65.7,61,2.26,46,1.70,29.6,6.70,0.31,37.0,2024
2,3,Arsenal,27,19,4,4,68,23,45,61,2.26,55.3,18.4,36.9,1.37,60204,Bukayo Saka - 13,David Raya,NaN,61.1,65,2.41,45,1.67,40.7,5.48,0.39,32.7,2024
3,4,Aston Villa,27,17,4,6,59,37,22,55,2.04,50.2,38.3,11.9,0.44,41651,Ollie Watkins - 16,Emiliano Martínez,NaN,55.6,56,2.07,41,1.52,22.2,5.41,0.36,37.1,2024
4,5,Tottenham,26,15,5,6,55,39,16,50,1.92,45.4,43.0,2.4,0.09,61555,Son Heung-min - 13,Guglielmo Vicario,NaN,60.5,52,2.00,43,1.65,19.2,5.73,0.34,36.5,2024


In [8]:
overall_stats_df.shape

(120, 29)

In [9]:
overall_stats_df.to_csv("../datasets/overall_stats2.csv", index=False)